In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

path = "./MiniCPM3-4B"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map=device, trust_remote_code=True)

messages = [
    {"role": "user", "content": "推荐5个北京的景点。"},
]
model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)

model_outputs = model.generate(
    model_inputs,
    max_new_tokens=1024,
    top_p=0.7,
    temperature=0.7
)

output_token_ids = [
    model_outputs[i][len(model_inputs[i]):] for i in range(len(model_inputs))
]

responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
print(responses)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


北京，这座古老而又现代的城市，拥有着丰富的历史文化遗产和现代都市的繁华。以下是我为您推荐的五个不可错过的北京景点：

1. **故宫博物院**：作为世界上最大的古代宫殿建筑群，故宫不仅是中国古代皇家生活的象征，也是中国五千年文明史的缩影。在这里，您可以欣赏到精美的古代建筑和珍贵的文物藏品，感受中国古代皇家的奢华与庄重。

2. **颐和园**：这座皇家园林是中国保存最完整的皇家园林，集山水、建筑、园林艺术于一体，展现了中国古代园林艺术的精华。漫步在园中，可以感受到宁静与自然的和谐之美。

3. **天坛公园**：作为明清两代皇帝祭天祈谷的场所，天坛以其宏伟的建筑和深厚的文化底蕴吸引着无数游客。祈年殿的精美构造和古老的祭天仪式，让人仿佛穿越回古代，体验皇家祭祀的庄严与神圣。

4. **天安门广场**：作为世界上最大的城市中心广场，天安门广场不仅是中国的政治中心，也是国家的象征。在这里，您可以近距离观看人民英雄纪念碑、毛主席纪念堂等标志性建筑，感受中国历史的厚重与国家的尊严。

5. **长城**：作为世界文化遗产，长城是中华民族的象征，也是世界建筑史上的奇迹。八达岭长城以其雄伟壮观的景色和深厚的历史文化，吸引了无数游客前来观光。站在长城之上，可以感受到中华民族的坚韧与不屈。

这五个景点不仅展现了北京的历史文化底蕴，也体现了北京的自然风光与现代都市的魅力。无论是历史文化爱好者，还是自然风光追求者，都能在北京找到属于自己的精彩。


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def initialize_MiniCPM3(path, device):
    tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map=device, trust_remote_code=True)
    return tokenizer, model

def get_response(tokenizer, model, device, query):
    messages = [
        {"role": "user", "content": query},
    ]
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
    
    model_outputs = model.generate(
        model_inputs,
        max_new_tokens=1024,
        top_p=0.7,
        temperature=0.7
    )
    
    output_token_ids = [
        model_outputs[i][len(model_inputs[i]):] for i in range(len(model_inputs))
    ]
    
    responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
    return responses

def check_response_correctness(tokenizer, model, device, query, response, answer):
    prompt = f"Determine if the following response is correct based on the given answer:\nQuery: {query}\nResponse: {response}\nAnswer: {answer}\nPlease answer True or False."
    model_response = get_response(tokenizer, model, device, prompt)
    return model_response.strip().lower() == "true"

def main():
    path = "./MiniCPM3-4B"
    device = "cuda"
    
    tokenizer, model = initialize_MiniCPM3(path, device)
    
    query = "推荐5个北京的景点。"
    response = "天安门广场, 故宫, 颐和园, 天坛, 长城"
    answer = "天安门广场, 故宫, 颐和园, 天坛, 长城"
    is_correct = check_response_correctness(tokenizer, model, device, query, response, answer)
    print(is_correct)

if __name__ == "__main__":
    main()